# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [101]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados


(27028, 69)


(27028, 69)

In [54]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [55]:
# 3) seu código aqui
sinasc1 = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

sinasc1.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [65]:
# 4) seu código aqui

sinasc1['APGAR5'].fillna(11, inplace=True)
sinasc1 = sinasc1.loc[sinasc1["APGAR5"] != 11]
sinasc1.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
ASFIXIA       0
dtype: int64

In [57]:
# 5) seu código aqui
sinasc1['ESTCIVMAE'].fillna(9, inplace=True)
sinasc1['CONSULTAS'].fillna(9, inplace=True)
sinasc1[['ESTCIVMAE', 'CONSULTAS']]

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [58]:
# 6) Seu código aqui
sinasc1['QTDFILVIVO'].fillna(0, inplace=True)
sinasc1.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [59]:
# 7) seu código aqui

sinasc1['ESCMAE'].fillna('Nenhuma', inplace=True)
# Escolhi colocar "Nenhuma" como o nivel de escolaridade para os faltantes pois é bem provável que ficou sem preenchimento 
# por não ser nenhum mesmo.

sinasc1['GRAVIDEZ'].fillna('Única', inplace=True)
# Para mim também faz sentido preencher o que não foi preenchido em Gravidez, como sendo a única ou a primeira gravidez.

sinasc1['GESTACAO'].fillna(9, inplace=True)
sinasc1 = sinasc1.loc[sinasc1['GESTACAO'] != 9]
sinasc1.isna().sum()
# Acho que eliminar as linhas de dados faltando em 'GESTACAO' foi o mais acertado pois é um valor que pode afetar os dados.
# Nâo dá para dizer a quantas semanas de gestação está e nem estimar se não houver informação.

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [73]:
# 8) seu código aqui 'APGAR5'
# Entre 8 e 10 está em uma faixa 'normal'.
# Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
# Entre 4 e 5 significa 'asfixia moderada'.
# Entre 0 e 3 significa 'asfixia severa'.

sinasc1.loc[sinasc1['APGAR5'] >=8, 'ASFIXIA'] = 'Normal'
sinasc1.loc[(sinasc1['APGAR5'] <=7) & (sinasc1['APGAR5'] >=6), 'ASFIXIA'] = 'Asfixia leve'
sinasc1.loc[(sinasc1['APGAR5'] <=5) & (sinasc1['APGAR5'] >=4), 'ASFIXIA'] = 'Asfixia moderada'
sinasc1.loc[sinasc1['APGAR5'] <= 3, 'ASFIXIA'] = 'Asfixia severa'
round(sinasc1['ASFIXIA'].value_counts() / 26925 * 100, 2)


ASFIXIA
Normal              93.87
Asfixia leve         1.14
Asfixia severa       0.25
Asfixia moderada     0.23
Name: count, dtype: float64

In [109]:
# 9) seu código aqui capitalize()

sinasc1.loc[sinasc1['ASFIXIA'].eq("Asfixia leve")] = "Asfixia_Leve"
sinasc1.loc[sinasc1['ASFIXIA'].eq("Asfixia moderada")] = "Asfixia_Moderada"
sinasc1.loc[sinasc1['ASFIXIA'].eq("Asfixia severa")] = "Asfixia_Severa"
sinasc1['ASFIXIA'].head(30)


0             Normal
1             Normal
2             Normal
3             Normal
4             Normal
5             Normal
6             Normal
7             Normal
8             Normal
9             Normal
10            Normal
11            Normal
12            Normal
13            Normal
14      Asfixia_Leve
15            Normal
16            Normal
17            Normal
18            Normal
19            Normal
20    Asfixia_Severa
21            Normal
22            Normal
23            Normal
24            Normal
25            Normal
26            Normal
27            Normal
28            Normal
29            Normal
Name: ASFIXIA, dtype: object